In [1]:
import numpy as np
from astropy.table import Table as t
import os

from astropy.cosmology import Planck18 as cosmo
import astropy.units as u
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi'] = 360
matplotlib.rcParams['text.usetex'] = True
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']

plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 12,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
})

In [3]:
# URL base de los datos
base_url = 'https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/'

# Data
data_N = t.read(f'{base_url}BGS_ANY_N_clustering.dat.fits')
data_S = t.read(f'{base_url}BGS_ANY_S_clustering.dat.fits')

# Random
rand_N = t.read(f'{base_url}BGS_ANY_N_2_clustering.ran.fits')
rand_S = t.read(f'{base_url}BGS_ANY_S_2_clustering.ran.fits')

print("Datos cargados correctamente:")
print(f"→ data_N: {len(data_N)} entradas")
print(f"→ data_S: {len(data_S)} entradas")
print(f"→ rand_N: {len(rand_N)} entradas")
print(f"→ rand_S: {len(rand_S)} entradas")


Datos cargados correctamente:
→ data_N: 113691 entradas
→ data_S: 128055 entradas
→ rand_N: 215178 entradas
→ rand_S: 218397 entradas


In [4]:
from astropy.table import Table as t
import numpy as np
import os
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

# Carpeta de salida absoluta
output_dir = "../XYZ_outputs"
os.makedirs(output_dir, exist_ok=True)

north_rosetas = [3, 6, 7, 11, 12, 13, 14, 15, 18, 19]

# Función de conversión
def spherical_to_cartesian(ra, dec, z):
    d_z = cosmo.comoving_distance(z).to(u.Mpc).value  # comoving distance
    ra_rad = np.radians(ra)
    dec_rad = np.radians(dec)
    x = d_z * np.cos(dec_rad) * np.cos(ra_rad)
    y = d_z * np.cos(dec_rad) * np.sin(ra_rad)
    z = d_z * np.sin(dec_rad)
    return x, y, z

# Loop para exportar archivos por roseta
for rosette in north_rosetas:
    rosette_dat = data_N[data_N['ROSETTE_NUMBER'] == rosette]
    rosette_rand = rand_N[rand_N['ROSETTE_NUMBER'] == rosette]

    if len(rosette_dat) == 0 and len(rosette_rand) == 0:
        print(f"Roseta {rosette} sin datos en N")
        continue

    x_d, y_d, z_d = spherical_to_cartesian(rosette_dat['RA'], rosette_dat['DEC'], rosette_dat['Z'])
    x_r, y_r, z_r = spherical_to_cartesian(rosette_rand['RA'], rosette_rand['DEC'], rosette_rand['Z'])

    tab_data = t([rosette_dat['TARGETID'], x_d, y_d, z_d], names=('TARGETID','X','Y','Z'))
    tab_rand = t([rosette_rand['TARGETID'], x_r, y_r, z_r], names=('TARGETID','X','Y','Z'))

    tab_data.write(os.path.join(output_dir, f"BGS_{rosette}_clustering_data.ecsv"), format='ascii.ecsv', overwrite=True)
    tab_rand.write(os.path.join(output_dir, f"BGS_{rosette}_clustering_rand.ecsv"), format='ascii.ecsv', overwrite=True)

    print(f"Guardado: BGS_{rosette}_clustering_data.ecsv y rand.ecsv")

print("\n Vista previa de BGS_3_clustering_data:")
print(tab_data[:5])


✅ Guardado: BGS_3_clustering_data.ecsv y rand.ecsv
✅ Guardado: BGS_6_clustering_data.ecsv y rand.ecsv
✅ Guardado: BGS_7_clustering_data.ecsv y rand.ecsv
✅ Guardado: BGS_11_clustering_data.ecsv y rand.ecsv
✅ Guardado: BGS_12_clustering_data.ecsv y rand.ecsv
✅ Guardado: BGS_13_clustering_data.ecsv y rand.ecsv
✅ Guardado: BGS_14_clustering_data.ecsv y rand.ecsv
✅ Guardado: BGS_15_clustering_data.ecsv y rand.ecsv
✅ Guardado: BGS_18_clustering_data.ecsv y rand.ecsv
✅ Guardado: BGS_19_clustering_data.ecsv y rand.ecsv

📌 Vista previa de BGS_3_clustering_data:
     TARGETID              X                  Y                  Z         
                          deg                deg                deg        
----------------- ------------------- ------------------ ------------------
39633166880737301  -531.7459779774897 -827.4676908373093   981.187323853201
39633166855572110  -609.7170958098015 -877.0661073286476 1067.5668607084199
39633171146343696  -64.60450819437581 -97.11148698560076 117.

In [5]:
from astropy.table import Table as t
import numpy as np
import os
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

output_dir = "../XYZ_outputs"
os.makedirs(output_dir, exist_ok=True)

# Función de conversión esférica → cartesiana (usando distancia comóvil)
def spherical_to_cartesian(ra, dec, z):
    d_z = cosmo.comoving_distance(z).to(u.Mpc).value  # comoving distance
    ra_rad = np.radians(ra)
    dec_rad = np.radians(dec)
    x = d_z * np.cos(dec_rad) * np.cos(ra_rad)
    y = d_z * np.cos(dec_rad) * np.sin(ra_rad)
    z = d_z * np.sin(dec_rad)
    return x, y, z

south_rosetas = [0, 1, 2, 4, 5, 8, 9, 10, 16, 17]

for rosette in south_rosetas:
    rosette_dat = data_S[data_S['ROSETTE_NUMBER'] == rosette]
    rosette_rand = rand_S[rand_S['ROSETTE_NUMBER'] == rosette]

    if len(rosette_dat) == 0 and len(rosette_rand) == 0:
        print(f"Roseta {rosette} sin datos en S")
        continue

    x_d, y_d, z_d = spherical_to_cartesian(rosette_dat['RA'], rosette_dat['DEC'], rosette_dat['Z'])
    x_r, y_r, z_r = spherical_to_cartesian(rosette_rand['RA'], rosette_rand['DEC'], rosette_rand['Z'])

    tab_data = t([rosette_dat['TARGETID'], x_d, y_d, z_d], names=('TARGETID','X','Y','Z'))
    tab_rand = t([rosette_rand['TARGETID'], x_r, y_r, z_r], names=('TARGETID','X','Y','Z'))

    tab_data.write(os.path.join(output_dir, f"BGS_{rosette}_clustering_data.ecsv"), format='ascii.ecsv', overwrite=True)
    tab_rand.write(os.path.join(output_dir, f"BGS_{rosette}_clustering_rand.ecsv"), format='ascii.ecsv', overwrite=True)

    print(f"Guardado: BGS_{rosette}_clustering_data.ecsv y rand.ecsv")

print("\n Vista previa de BGS_0_clustering_data:")
print(tab_data[:5])


Guardado: BGS_0_clustering_data.ecsv y rand.ecsv
Guardado: BGS_1_clustering_data.ecsv y rand.ecsv
Guardado: BGS_2_clustering_data.ecsv y rand.ecsv
Guardado: BGS_4_clustering_data.ecsv y rand.ecsv
Guardado: BGS_5_clustering_data.ecsv y rand.ecsv
Guardado: BGS_8_clustering_data.ecsv y rand.ecsv
Guardado: BGS_9_clustering_data.ecsv y rand.ecsv
Guardado: BGS_10_clustering_data.ecsv y rand.ecsv
Guardado: BGS_16_clustering_data.ecsv y rand.ecsv
Guardado: BGS_17_clustering_data.ecsv y rand.ecsv

 Vista previa de BGS_0_clustering_data:
     TARGETID             X                   Y                  Z         
                         deg                 deg                deg        
----------------- ------------------ ------------------- ------------------
39627812469999531 -383.3577234129593 -254.59384036800634   7.08128215109759
39627812453221848 -811.8954107066853  -518.5909908797123  18.20997911162563
39627806430202635  -912.235751949022  -606.9946387254172 16.721702887220786
3962780640